In [1]:
## CCW Implementation
## ((a1, a2), (b1, b2), (c1, c2)) -> int
def CCW(a,b,c):
    det = a[0]*b[1] - a[1]*b[0] + a[1]*c[0] - a[0]*c[1] + b[0]*c[1] - c[0]*b[1]
    if det < 0:
        return 1
    if det > 0:
        return -1
    if (a == b):
        if a == c:
            return 0
        return 2
    if (a[0] <= b[0] and b[0] <= c[0]) or (a[0] >= b[0] and b[0] >= c[0]):
        return 2
    if (a[0] <= c[0] and c[0] <= b[0]) or (a[0] >= c[0] and c[0] >= b[0]):
        return 0
    else:
        return -2

In [2]:
## (((p1,p2), (q1,q2)), ((a1,a2), (b1,b2)) -> bool
def Intersect(l1, l2):
    if l1[0] == l1[1] and abs(CCW(*l2, l1[0])) == 1:
        return False
    if l2[0] == l2[1] and abs(CCW(*l1, l2[0])) == 1:
        return False
    if l2[0] == l2[1] and l1[0] == l1[1] and l2[0] != l1[1]:
        return False
    elif l2[0] == l2[1] and l1[0] == l1[1]:
        return True
    if abs(CCW(*l1, l2[0])) != 1 or abs(CCW(*l1, l2[1])) != 1 or abs(CCW(*l2, l1[0])) != 1 or abs(CCW(*l2, l1[1])) != 1:
        return True
    first = CCW(*l1, l2[0]) == -1*CCW(*l1, l2[1])

    second = CCW(*l2, l1[0]) == -1*CCW(*l2, l1[1])

    return first and second

In [3]:
#Pass in Array of 2-tuples
#call x.hull to get the convex hull
class QuickHull:
    def __init__(self, points):
        points = set(points)
        if len(points) <= 2:
            self.hull = points
            return
        if len(points) == 3:
            points = list(points)
            if CCW(points[0], points[1], points[2]) != 2 and CCW(points[0], points[1], points[2]) != 0 and CCW(points[0], points[1], points[2]) != -2:
                self.hull = set(points)
            else:
                self.hull = set([points[0], points[1]])
            return
        r, l = (max(points), min(points))
        under = []
        over = []
        for i in points:
            if CCW(i, r, l) == -1:
                under.append(i)
            else:
                over.append(i)
        upperhull = set(self.construct(under, l, r))
        lowerhull = set(self.construct(over, r, l))
        self.hull = upperhull | lowerhull
        
    def findmax(self, s, p1, p2):
        maxx = s[0]
        cur = self.dist(maxx, p1, p2)
        for i in s[1:]:
            if self.distanceComp(self.dist(i, p1, p2), cur):
                maxx = i
                cur = self.dist(i, p1, p2)
        return maxx
    
    def dist(self, q, p1, p2):
        # (distance formula derivation using projections)
        n = pow((p2[1] - p1[1])*q[0] - (p2[0] - p1[0])*q[1] + p2[0]*p1[1] - p2[1]*p1[0], 2)
        m = (p2[1] - p1[1])*(p2[1] - p1[1]) + (p2[0] - p1[0])*(p2[0] - p1[0])
        return (n,m)
    
    def distanceComp(self, d1, d2):
        return d1[0]*d2[1] > d1[1]*d2[0]
    
    def prune(self, s, p1, p2, p_):
        s1 = []
        s2 = []
        for i in s:
            if not self.triangle(i, p1, p_, p2):
                if not self.distanceComp(self.dist(i, p1, p_), self.dist(i, p_, p2)):
                    s1.append(i)
                else:
                    s2.append(i)
        return (s1, s2)
        
        
    def triangle(self, q, p1, p2, p3):
        a = CCW(q, p1, p2)
        b = CCW(q, p2, p3)
        if (abs(a) != 1 or abs(b) != 1):
            return True
        if (a == b):
            return True
        return False
            
        
    def construct(self,pts, q1, q2):
        acc = [q1, q2]
        def helper(s, p1, p2):
            if s == []:
                return
            p_ = self.findmax(s, p1, p2)
            acc.append(p_)
            s1, s2 = self.prune(s, p1, p2, p_)
            helper(s1, p1, p_)
            helper(s2, p_, p2)
        helper(pts, q1, q2)
        return acc


In [4]:
import matplotlib.pyplot as plt
class Onion:
    def __init__(self, points):
        self.points = set(points)
        self.onionLayers = []
    def peelLayer(self, display = True):
        if len(self.points) == 0:
            print("peeled")
            return False
        x = QuickHull(self.points).hull
        self.onionLayers.append(x)
        self.display(x, display)
        self.points = self.points - x
        return True
    def display(self, pts, disp):
        if not disp:
            return
        x = []
        y = []
        for i in self.points:
            x.append(i[0])
            y.append(i[1])
        plt.plot(x,y, 'ro')
        _x = []
        _y = []
        for i in pts:
            _x.append(i[0])
            _y.append(i[1])
        plt.plot(x,y, 'ro')
        plt.plot(_x,_y, 'bo')
        plt.show()
    def peelAll(self):
        while(self.peelLayer()):
            pass
        return self.numberLayers()
    def silentPeelAll(self):
        while(self.peelLayer(display = False)):
            pass
        return self.numberLayers()
    def numberLayers(self):
        return len(self.onionLayers)

In [5]:
import random
import math
class randgrid:
    def circle(self,number, radius):
        points = []
        for i in range(number):
            r = math.sqrt(random.uniform(0, radius))
            theta = random.uniform(0, 2*math.pi)
            points.append((r *math.cos(theta), r*math.sin(theta)))
        return points
    def square(self,number, side):
        points = []
        for i in range(number):
            points.append((random.uniform(0, side), random.uniform(0,side)))
        return points


In [6]:
class testHarness:
    def measureCircle(self, n):
        print(Onion(randgrid().circle(n,1)).silentPeelAll())
    def measureSquare(self, n):
        print(Onion(randgrid().square(n,1)).silentPeelAll())

In [8]:
testHarness().measureSquare(10000)

peeled
240
